In [ ]:
!pip install yfinance

In [ ]:
import requests
import pandas as pd
import yfinance as yf
import numpy as np

API KEY y TICKER

In [ ]:
# Solicita al usuario los datos necesarios
api_key = "N8C0FFUG4981TUB5"
#api_key = input("Introduce tu API key de Alpha Vantage: ").strip()
symbol = input("Introduce el ticker de la acción (ejemplo: AAPL): ").strip().upper()

EXTRACCIÓN DE DATOS HISTÓRICOS DE LA ACCIÓN

In [ ]:
# Consulta SIEMPRE la serie diaria NO ajustada y con máximo histórico posible
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={api_key}'

r = requests.get(url)
data = r.json()

# Chequear errores comunes en la respuesta
if "Information" in data:
    print(f"Alpha Vantage dice: {data['Information']}")
    exit()
if "Note" in data:
    print(f"Límite de Alpha Vantage alcanzado: {data['Note']}")
    exit()
if "Error Message" in data:
    print(f"Error de Alpha Vantage: {data['Error Message']}")
    exit()

# Busca la clave de la serie temporal
series_key = next((k for k in data.keys() if "Time Series" in k), None)
if not series_key:
    print("No se encontró la serie temporal. Respuesta de Alpha Vantage:")
    print(data)
    exit()

df = pd.DataFrame.from_dict(data[series_key], orient="index")
df.index = pd.to_datetime(df.index)
df = df.sort_index()

# Usamos el precio de cierre NO ajustado
if '4. close' in df.columns:
    df = df[['4. close']].rename(columns={'4. close': 'Close'})
else:
    print("No se encontró la columna '4. close'. Columnas disponibles:")
    print(df.columns)
    exit()

In [ ]:
print(df.head())

EXTRACCIÓN DE DATOS HISTÓRICOS DEL ÍNDICE

In [ ]:
# Mapeo de mercados a índices principales en Yahoo Finance
MERCADO_A_INDICE = {
    "NASDAQ": "^IXIC",     # NASDAQ Composite
    "NYSE": "^NYA",        # NYSE Composite
    "S&P500": "^GSPC",     # S&P 500
    "DOWJONES": "^DJI",    # Dow Jones Industrial Average
    "BME": "^IBEX",        # IBEX 35 (España)
    "FTSE": "^FTSE",       # FTSE 100 (UK)
    "HSI": "^HSI",         # Hang Seng (Hong Kong)
    # Agrega otros mercados e índices según necesidad
}

# Usuario ingresa el mercado
mercado = input("¿En qué mercado cotiza la acción? (Ejemplo: NASDAQ, NYSE, S&P500, BME): ").strip().upper().replace(" ", "")

indice = MERCADO_A_INDICE.get(mercado)
if not indice:
    print("No se encontró índice para ese mercado. Símbolos soportados:")
    for m, idx in MERCADO_A_INDICE.items():
        print(f"{m}: {idx}")
    exit()

LIMPIEZA DE DATOS DEL ÍNDICE POR FECHAS

In [ ]:
# OBTIENE RANGO DE FECHAS DE TU DATAFRAME df
fecha_inicio = df.index.min()
fecha_fin = df.index.max()

# DESCARGA EL ÍNDICE SOLO EN EL RANGO DE FECHAS
indice_df = yf.download(indice, start=str(fecha_inicio.date()), end=str(fecha_fin.date()))

if 'Close' in indice_df.columns:
    indice_df = indice_df[['Close']]
else:
    print("No se encontró la columna 'Close' en el índice. Columnas disponibles:")
    print(indice_df.columns)
    exit()

In [ ]:
print(indice_df.head())

FILTRADO DE DATAFRAMES

In [ ]:
# Unimos ambos DataFrames por la fecha (índice), dejando solo las fechas comunes
df_accion_filtrado, df_indice_filtrado = df.align(indice_df, join='inner', axis=0)

# Convertir a float antes de calcular retornos
col = [c for c in df_indice_filtrado.columns if c[0] == 'Close'][0]
df_indice_filtrado[col] = pd.to_numeric(df_indice_filtrado[col], errors='coerce')
df_accion_filtrado['Close'] = pd.to_numeric(df_accion_filtrado['Close'], errors='coerce')

print("Fechas coincidentes:", df_accion_filtrado.shape[0])
print(df_accion_filtrado.head())
print(df_indice_filtrado.head())

CALCULO DE DESVIO ESTANDAR

In [ ]:
# Calcula retornos logarítmicos diarios
ret_accion = np.log(df_accion_filtrado['Close'] / df_accion_filtrado['Close'].shift(1)).dropna()
ret_indice = np.log(df_indice_filtrado[col] / df_indice_filtrado[col].shift(1)).dropna()

# Alinear retornos (por si pierden una fecha con el shift)
ret_accion, ret_indice = ret_accion.align(ret_indice, join='inner')

# Desvío estándar de la acción (volatilidad)
desvio_accion = ret_accion.std()

# Expresado en porcentaje (%)
desvio_accion_pct = desvio_accion * 100

print(f"Desvío estándar de la acción (porcentaje): {desvio_accion_pct:.4f}%")

CALCULO DE COVARIANZA

In [ ]:
# Covarianza acción-índice
covarianza = np.cov(ret_accion, ret_indice)[0,1]
print(f"Covarianza acción-índice: {covarianza:.6f}")

CALCULO DE INDICE DE CORRELACION

In [ ]:
# Covarianza acción-índice
covarianza = np.cov(ret_accion, ret_indice)[0,1]
print(f"Covarianza acción-índice: {covarianza:.6f}")

# Índice de correlación (coeficiente de correlación de Pearson)
correlacion = np.corrcoef(ret_accion, ret_indice)[0,1]
print(f"Índice de correlación acción-índice: {correlacion:.6f}")

CALCULO DE BETA

In [ ]:
# Beta = Cov(accion, indice) / Var(indice)
beta = covarianza / ret_indice.var()
print(f"Beta de la acción respecto al índice: {beta:.4f}")

In [ ]:
# Asegurarse que el índice está en datetime
df_accion_filtrado.index = pd.to_datetime(df_accion_filtrado.index)

# Calcular retornos logarítmicos diarios
ret_accion = np.log(df_accion_filtrado['Close'] / df_accion_filtrado['Close'].shift(1))

# Agrupar por año y mes, y calcular la varianza de los retornos diarios en cada mes
varianza_mensual = ret_accion.groupby([ret_accion.index.year, ret_accion.index.month]).var()

varianza_mensual.index = [f"{a}-{m:02d}" for a, m in varianza_mensual.index]
varianza_mensual_df = varianza_mensual.to_frame(name='varianza_mensual')

promedio_varianza_mensual = varianza_mensual_df['varianza_mensual'].mean()

promedio_varianza_mensual_pct2 = promedio_varianza_mensual * 10000
print(f"Varianza mensual promedio (%^2): {promedio_varianza_mensual_pct2:.6f}")